In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
def run_ex1_44_full(initial_lr, iterations, lambda_reg = 0.001, beta1 = 0.9, beta2 = 0.999, batch_size = 128, epsilon = 1e-8, master_seed = 42, print_every = 50, verbose = True):
    np.random.seed(master_seed)
    n_samples = 500
    area = np.random.uniform(10, 50, n_samples)
    expected_bedrooms = np.clip((area/15), 0, 4)
    bedrooms = np.random.normal(expected_bedrooms, 0.5)
    bedrooms = np.round(bedrooms).astype(int)
    bedrooms = np.clip(bedrooms, 0, 4)
    age = np.random.uniform(0, 30, n_samples)
    noise = np.random.normal(0, 2 , n_samples)
    rent = 1.5 * area + 2 * bedrooms - 0.8 * age + 5 + noise

    X = np.column_stack((area, bedrooms, age))
    y = rent

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = master_seed)
    scaler = StandardScaler()
    X_train_standardized = scaler.fit_transform(X_train)
    X_test_standardized = scaler.transform(X_test)

    mean = scaler.mean_
    scale = scaler.scale_
    print('mean = ', np.round(mean, 4))
    print('scale = ', np.round(scale, 4))

    w = np.random.rand(X_train_standardized.shape[1])
    b = np.random.rand()

    m_w = np.zeros_like(w)
    m_b = 0
    v_w = np.zeros_like(w)
    v_b = 0

    lr_history = []
    loss_history_train = []
    loss_history_test = []
    w_history = []
    b_history = []

    for i in range(iterations):
        np.random.seed(master_seed + i)
        permutation = np.random.permutation(len(X_train_standardized))
        X_shuffled = X_train_standardized[permutation]
        y_shuffled = y_train[permutation]

        for start_idx in range(0, len(X_train_standardized), batch_size):
            end_idx = start_idx + batch_size
            X_batch = X_shuffled[start_idx:end_idx]
            y_batch = y_shuffled[start_idx:end_idx]

            y_pred_batch = np.dot(X_batch, w) + b
            error_batch = y_batch - y_pred_batch

            w_gradient = (-2/ len(X_batch)) * np.dot(X_batch.T, error_batch) + 2 * lambda_reg * w
            b_gradient = (-2/ len(X_batch)) * np.sum(error_batch)

            m_w = beta1 * m_w + (1 - beta1) * w_gradient
            m_b = beta1 * m_b + (1 - beta1) * b_gradient
            v_w = beta2 * v_w + (1 - beta2) * (w_gradient ** 2)
            v_b = beta2 * v_b + (1 - beta2) * (b_gradient ** 2)

            m_w_hat = m_w / (1 - beta1 ** (i+1))
            m_b_hat = m_b / (1 - beta1 ** (i+1))
            v_w_hat = v_w / (1 - beta2 ** (i+1))
            v_b_hat = v_b / (1 - beta2 ** (i+1))

            w -= initial_lr * m_w_hat / (np.sqrt(v_w_hat) + epsilon)
            b -= initial_lr * m_b_hat / (np.sqrt(v_b_hat) + epsilon)

            w_history.append(w.copy())
            b_history.append(b)
            lr_history.append(initial_lr)
        
        y_pred_train = np.dot(X_train_standardized, w) + b
        error_train = y_train - y_pred_train
        loss_train = np.mean(error_train ** 2) + lambda_reg * np.sum(w **2)
        loss_history_train.append(loss_train)

        y_pred_test = np.dot(X_test_standardized, w) + b
        error_test = y_test - y_pred_test
        loss_test = np.mean(error_test ** 2)
        loss_history_test.append(loss_test)

        if verbose and (i % print_every == 0 or i == iterations-1):
            print(f'Iteration {i} : w = {np.round(w, 4)}, b = {b:.4f}, Train Loss = {loss_train:.4f}, Test Loss = {loss_test:.4f}')
    
    loss_history_test_arr = np.array(loss_history_test)
    min_index = np.argmin(loss_history_test_arr)
    min_test_loss = loss_history_test_arr[min_index]
    best_w = w_history[min_index]
    best_b = b_history[min_index]
    lr_at_best = lr_history[min_index]

    df_summary = pd.DataFrame({
        'Final Train Loss' : [np.round(loss_history_train[iterations-1], 4)],
        'Final Test Loss' : [np.round(loss_history_test[iterations-1], 4)],
        'Best Iteration' : [min_index],
        'Train Loss @ Best Test' : [np.round(loss_history_train[min_index], 4)],
        'Best Test Loss' : [np.round(min_test_loss, 4)],
        'w (params)' : [np.round(best_w, 4)],
        'b (bias)' : [np.round(best_b, 4)],
        'Learning Rate @ Best Test' : [np.round(lr_at_best, 4)]
    })

    return min_test_loss, min_index, best_w, best_b, df_summary, loss_history_train, loss_history_test, lr_history

min_test_loss, min_index, best_w, best_b, df_summary, loss_history_train, loss_history_test, lr_history = run_ex1_44_full(iterations = 600, initial_lr = 4.84, master_seed = 42, verbose = True)
display(df_summary)

def lr_grid_search_ex1_44(lr_start = 0.2, lr_end = 8, lr_step = 0.01, iterations = 600, master_seed = 42):
    results = []
    best_lr = None
    best_loss = np.inf

    lr_list = np.arange(lr_start, lr_end + lr_step, lr_step)

    for lr in lr_list:
        print('Testing LR = ', np.round(lr, 2))

        min_test_loss, min_index, best_w, best_b, df_summary, loss_history_train, loss_history_test, lr_history = run_ex1_44_full(iterations = iterations, initial_lr = lr, master_seed = master_seed, verbose = False)
        results.append({'lr' : lr, 'best_test_loss' : np.round(min_test_loss, 4)})

        if min_test_loss < best_loss:
            best_loss = min_test_loss
            best_lr = lr

    df = pd.DataFrame(results).sort_values('best_test_loss').reset_index(drop = True)
    return best_lr, df

        

mean =  [30.1526  1.99   14.6872]
scale =  [11.9856  1.0246  8.5246]
Iteration 0 : w = [ 23.8029  20.6717 -11.3211], b = 26.8133, Train Loss = 828.6490, Test Loss = 843.2399
Iteration 50 : w = [18.0549  2.1329 -6.8195], b = 42.7170, Train Loss = 3.9147, Test Loss = 5.5643
Iteration 100 : w = [17.7119  2.2239 -7.0903], b = 42.6405, Train Loss = 3.9734, Test Loss = 5.6886
Iteration 150 : w = [17.84    2.1345 -6.9048], b = 42.5703, Train Loss = 3.9111, Test Loss = 5.6094
Iteration 200 : w = [17.7246  2.5112 -7.1203], b = 42.4551, Train Loss = 4.0380, Test Loss = 5.6039
Iteration 250 : w = [17.8657  2.1833 -6.6766], b = 42.7951, Train Loss = 3.9334, Test Loss = 5.7456
Iteration 300 : w = [18.0582  2.2127 -6.7679], b = 42.7081, Train Loss = 3.9367, Test Loss = 5.5711
Iteration 350 : w = [17.4957  2.5508 -6.7284], b = 42.6372, Train Loss = 3.9246, Test Loss = 5.7236
Iteration 400 : w = [17.589   2.09   -7.1684], b = 43.1650, Train Loss = 4.4136, Test Loss = 6.3077
Iteration 450 : w = [17.833

,Final Train Loss,Final Test Loss,Best Iteration,Train Loss @ Best Test,Best Test Loss,w (params),b (bias),Learning Rate @ Best Test
0,4.9624,6.5773,463,3.9586,5.4962,"[17.6777, 2.3817, -6.8499]",42.5826,4.84


In [3]:
best_lr, df_lr = lr_grid_search_ex1_44(lr_start = 0.2, lr_end = 8, lr_step = 0.01, iterations = 600, master_seed = 42)
print('找到 Grid Search 最佳的 LR = ', np.round(best_lr, 4))
display(df_lr.head(10))

Testing LR =  0.2
mean =  [30.1526  1.99   14.6872]
scale =  [11.9856  1.0246  8.5246]
Testing LR =  0.21
mean =  [30.1526  1.99   14.6872]
scale =  [11.9856  1.0246  8.5246]
Testing LR =  0.22
mean =  [30.1526  1.99   14.6872]
scale =  [11.9856  1.0246  8.5246]
Testing LR =  0.23
mean =  [30.1526  1.99   14.6872]
scale =  [11.9856  1.0246  8.5246]
Testing LR =  0.24
mean =  [30.1526  1.99   14.6872]
scale =  [11.9856  1.0246  8.5246]
Testing LR =  0.25
mean =  [30.1526  1.99   14.6872]
scale =  [11.9856  1.0246  8.5246]
Testing LR =  0.26
mean =  [30.1526  1.99   14.6872]
scale =  [11.9856  1.0246  8.5246]
Testing LR =  0.27
mean =  [30.1526  1.99   14.6872]
scale =  [11.9856  1.0246  8.5246]
Testing LR =  0.28
mean =  [30.1526  1.99   14.6872]
scale =  [11.9856  1.0246  8.5246]
Testing LR =  0.29
mean =  [30.1526  1.99   14.6872]
scale =  [11.9856  1.0246  8.5246]
Testing LR =  0.3
mean =  [30.1526  1.99   14.6872]
scale =  [11.9856  1.0246  8.5246]
Testing LR =  0.31
mean =  [30.152

,lr,best_test_loss
0,4.85,5.4962
1,4.84,5.4962
2,4.83,5.4962
3,4.82,5.4963
4,4.86,5.4963
5,0.52,5.4965
6,4.87,5.4965
7,4.81,5.4965
8,0.51,5.4965
9,4.80,5.4967
